<a href="https://colab.research.google.com/github/priyal6/DL/blob/main/Pytorch_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import(
    DataLoader,
)
from tqdm import tqdm

In [2]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()

    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 3

#load data
train_dataset = datasets.MNIST(
    root = "dataset/", train=True, transform = transforms.ToTensor(), download= True
)

test_dataset = datasets.MNIST (
    root="dataset/", train = False, transform = transforms.ToTensor(), download = True
)

train_loader = DataLoader(dataset = train_dataset, batch_size=batch_size, shuffle =True)
test_loader = DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 15.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 425kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.02MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.03MB/s]


In [3]:
#initializing the network
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [4]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
#network
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    data = data.to(device=device)
    targets = targets.to(device=device)

    data = data.reshape(data.shape[0], -1)

    scores = model(data)
    loss = criterion(scores, targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()


100%|██████████| 938/938 [00:08<00:00, 106.27it/s]


In [7]:
def check_accuracy(loader, model):
    """
    Check accuracy of our trained model given a loader and a model

    Parameters:
        loader: torch.utils.data.DataLoader
            A loader for the dataset you want to check accuracy on
        model: nn.Module
            The model you want to check accuracy on

    Returns:
        acc: float
            The accuracy of the model on the dataset given by the loader
    """

    num_correct = 0
    num_samples = 0
    model.eval()

    # We don't need to keep track of gradients here so we wrap it in torch.no_grad()
    with torch.no_grad():
        # Loop through the data
        for x, y in loader:

            # Move data to device
            x = x.to(device=device)
            y = y.to(device=device)

            # Get to correct shape
            x = x.reshape(x.shape[0], -1)

            # Forward pass
            scores = model(x)
            _, predictions = scores.max(1)

            # Check how many we got correct
            num_correct += (predictions == y).sum()

            # Keep track of number of samples
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

In [8]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 92.21
Accuracy on test set: 92.49
